In [32]:
import pandas as pd
import numpy as np

# Read csv
listings_df = pd.read_csv('./data/with_amenities_and_estimated_income.csv', low_memory = False)

# Drop columns that aren't related to income or not feasible to capture from user
columns_to_drop = ['Unnamed: 0', 'id', 'scrape_id', 'host_id', 'host_total_listings_count',
                  'latitude', 'longitude', 'availability_30', 'availability_60', 'availability_90',
                  'availability_365', 'number_of_reviews', 'calculated_host_listings_count', 
                  'reviews_per_month', 'Other', 'listing_url', 'last_scraped', 'host_name',
                  'experiences_offered', 'picture_url', 'name', 'host_url', 'host_since',
                  'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_listings_count',
                  'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street',
                  'city', 'neighbourhood_group_cleansed', 'smart_location', 'country_code',
                  'country', 'is_location_exact', 'amenities', 'price', 'calendar_updated', 'has_availability',
                  'calendar_last_scraped', 'first_review', 'last_review', 'requires_license',
                  'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready',
                  'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification',
                  'translation missing: en.hosting_amenity_49', 'summary', 'space', 'description',
                  'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules',
                  'thumbnail_url', 'medium_url', 'xl_picture_url', 'host_location', 'host_about',
                  'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'state',
                  'neighbourhood_cleansed', 'host_neighbourhood', 'license', 'review_scores_rating',
                  'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                  'review_scores_communication', 'review_scores_location', 'review_scores_value',
                  'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'market']
for col in columns_to_drop:
    listings_df.drop([col], axis=1, inplace=True)

# Remove rows that don't have an estimated income per month
listings_df = listings_df[~pd.isna(listings_df['estimated_income_per_month'])]

# Dropping square feet because 7450 out of 7712 (97%) rows are null
listings_df.drop(['square_feet'], axis=1, inplace=True)

# Fill values going forward
listings_df.fillna(method ='ffill', inplace=True)

# Convert zipcode to string rather than float
listings_df['zipcode'] = listings_df['zipcode'].astype('int').astype('str')

# Convert $ amount for extra people from string to float
listings_df['extra_people'] = listings_df['extra_people'].apply(lambda s: s[1:]).astype('float')


In [33]:
listings_df = pd.get_dummies(listings_df)

estimated_income = listings_df['estimated_income_per_month']
listings_df = listings_df.drop(['estimated_income_per_month'], axis=1)

In [34]:
from sklearn.model_selection import train_test_split # splitting data

train_features, test_features, train_outcome, test_outcome = train_test_split(
    listings_df,
    estimated_income,
    test_size=0.20, 
    random_state=11
)


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

model = LinearRegression()
#backward selection
rfe = RFE(model, 100)
rfe = rfe.fit(train_features, train_outcome)
filterparam = rfe.support_

#finding the total columns
totalparams = train_features.columns
iqmodelparams = totalparams[filterparam]
iqmodelparams

Index(['accommodates', 'Outdoor seating', 'Pool', 'EV charger',
       'Warming drawer', 'Heat lamps', 'Rain shower', 'Sound system',
       'Building staff', 'Firm mattress', 'Projector and screen',
       'En suite bathroom', 'Shared gym', 'Bathroom essentials', 'Balcony',
       'Washer / Dryer', 'Waterfront', 'Mini fridge', 'Soaking tub',
       'Host greets you', 'Table corner guards', 'Beach view',
       'Shared hot tub', 'Hangers', 'Terrace', 'Disabled parking spot',
       'Doorman', 'Paid parking off premises', 'Netflix', 'Printer',
       'Breakfast table', 'Heated floors', 'Shared pool',
       'Stand alone steam shower', 'Ground floor access', 'Toilet paper',
       'Hot water kettle', 'Ski-in/Ski-out', 'Amazon Echo', 'Fax machine',
       'Baby monitor', 'Heated towel rack', 'Mountain view',
       'Formal dining area', 'Private hot tub', 'Standing valet',
       'Bedroom comforts', 'Day bed', 'Self check-in',
       'Bathtub with bath chair', 'Beachfront', 'Central air c

In [36]:
listings_df['estimated_income_per_month'] = estimated_income
top_features_df = listings_df[['accommodates', 'Outdoor seating', 'Pool', 'EV charger',
       'Warming drawer', 'Heat lamps', 'Rain shower', 'Sound system',
       'Building staff', 'Firm mattress', 'Projector and screen',
       'En suite bathroom', 'Shared gym', 'Bathroom essentials', 'Balcony',
       'Washer / Dryer', 'Waterfront', 'Mini fridge', 'Soaking tub',
       'Host greets you', 'Table corner guards', 'Beach view',
       'Shared hot tub', 'Hangers', 'Terrace', 'Disabled parking spot',
       'Doorman', 'Paid parking off premises', 'Netflix', 'Printer',
       'Breakfast table', 'Heated floors', 'Shared pool',
       'Stand alone steam shower', 'Ground floor access', 'Toilet paper',
       'Hot water kettle', 'Ski-in/Ski-out', 'Amazon Echo', 'Fax machine',
       'Baby monitor', 'Heated towel rack', 'Mountain view',
       'Formal dining area', 'Private hot tub', 'Standing valet',
       'Bedroom comforts', 'Day bed', 'Self check-in',
       'Bathtub with bath chair', 'Beachfront', 'Central air conditioning',
       'Body soap', 'Alfresco bathtub', 'Jetted tub', 'neighbourhood_Belltown',
       'neighbourhood_Brighton', 'neighbourhood_Cedar Park',
       'neighbourhood_Central Business District', 'neighbourhood_Fauntleroy',
       'neighbourhood_Industrial District',
       'neighbourhood_International District',
       'neighbourhood_Lower Queen Anne', 'neighbourhood_Madison Park',
       'neighbourhood_Pike Place Market', 'neighbourhood_Pioneer Square',
       'neighbourhood_Queen Anne', 'neighbourhood_Rainier Beach',
       'neighbourhood_Rainier View', 'neighbourhood_South Beacon Hill',
       'neighbourhood_South Lake Union', 'neighbourhood_Westlake',
       'neighbourhood_Yesler Terrace', 'zipcode_98052', 'zipcode_98101',
       'zipcode_98102', 'zipcode_98104', 'zipcode_98110', 'zipcode_98112',
       'zipcode_98122', 'zipcode_98134', 'zipcode_98155',
       'property_type_Apartment', 'property_type_Bed and breakfast',
       'property_type_Boat', 'property_type_Boutique hotel',
       'property_type_Camper/RV', 'property_type_Castle',
       'property_type_Condominium', 'property_type_Cottage',
       'property_type_Farm stay', 'property_type_Guest suite',
       'property_type_Hostel', 'property_type_House',
       'property_type_Houseboat', 'property_type_Serviced apartment',
       'property_type_Treehouse', 'property_type_Villa',
       'room_type_Entire home/apt', 'bed_type_Couch', 'estimated_income_per_month']].copy()


In [37]:
top_train_features, top_test_features, top_train_outcome, top_test_outcome = train_test_split(
    top_features_df.drop(['estimated_income_per_month'], axis=1),
    top_features_df['estimated_income_per_month'],
    test_size=0.20, 
    random_state=11
)
len(top_features_df.columns.values)

101

In [38]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import Pipeline                # for making pipelines
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

scaler = MinMaxScaler()
select = SelectPercentile()

# define a pipeline
pipe = Pipeline([('scaler', scaler), ('KNeighborsRegressor', KNeighborsRegressor())])

folds = KFold(n_splits = 10, shuffle = True)

# # defines a grid to search through
param_grid = {
#     'selectpercentile__percentile':range(10, 30, 5),
    'KNeighborsRegressor__n_neighbors': [3, 75, 211],
    'KNeighborsRegressor__weights':["uniform", "distance"]
}

# performs a grid search of pipeline
knngrid = GridSearchCV(pipe, param_grid,cv = folds, scoring="neg_mean_absolute_error")
knnwholemodel = knngrid.fit(top_train_features, top_train_outcome)
knnwholemodel.score(top_test_features, top_test_outcome)

-1064.0642737233477

In [217]:
knniqwholemodel.best_estimator_

Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('KNeighborsRegressor', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=75, p=2,
          weights='distance'))])

In [206]:
# selected_threshold = knniqwholemodel.best_estimator_.named_steps['threshold'].get_support()
# selected_threshold_columns = list()
# for i in range(len(selected_threshold)):
#     if (selected_threshold[i]):
#         selected_threshold_columns.append(test_features.columns.values[i])
        
# selected_threshold_columns

selected_columns = knniqwholemodel.best_estimator_.named_steps['selectpercentile'].get_support()
selected_k_best_columns = list()
for i in range(len(selected_columns)):
    if (selected_columns[i]):
        selected_k_best_columns.append(test_features.columns.values[i])

selected_k_best_columns

/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


['accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'guests_included',
 'minimum_nights',
 'Pool',
 'High chair',
 'Warming drawer',
 'Heat lamps',
 'Sound system',
 'Espresso machine',
 'Building staff',
 'Hair dryer',
 'Shampoo',
 'En suite bathroom',
 'Balcony',
 'Soaking tub',
 'Crib',
 'Table corner guards',
 'Beach view',
 'Wine cooler',
 'Smoke detector',
 'Hangers',
 'Terrace',
 'DVD player',
 'Hammock',
 'Heating',
 'Gas oven',
 'Printer',
 'Smart TV',
 'Elevator',
 'Buzzer/wireless intercom',
 'Outdoor parking',
 'Hot water kettle',
 'Walk-in shower',
 'Changing table',
 'Ceiling fan',
 'Ski-in/Ski-out',
 'Amazon Echo',
 'Private gym',
 'Baby monitor',
 'Formal dining area',
 'Bathtub with bath chair',
 'Private bathroom',
 'Hot water',
 'Beachfront',
 'Essentials',
 'Exercise equipment',
 'Gym',
 'Sun loungers',
 'neighbourhood_Arbor Heights',
 'neighbourhood_Belltown',
 'neighbourhood_Cedar Park',
 'neighbourhood_Central Business District',
 'neighbourhood_First Hill',
 '

In [207]:
knniqwholemodel.best_score_

-1306.9369057908384